<a href="https://colab.research.google.com/github/Ssactl/AH2179_Applied-AI-in-Transportation/blob/main/Module6_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data preparation

In [2]:
import pandas as pd
import numpy as np
#upload the dataset by downloading both datasets from canvas and upload it on colab
data_df = pd.read_csv("dataset_exercise_5_clustering_highway_traffic.csv",sep=";")
data_df

,PORTAL,Date,time_from,time_to,Interval_5,SPEED_MS_AVG,flow
0,"E4S 56,780",20210101,00:00:00,00:05:00,0,18.56,39
1,"E4S 56,780",20210101,00:05:00,00:10:00,1,20.39,18
2,"E4S 56,780",20210101,00:10:00,00:15:00,2,19.27,26
3,"E4S 56,780",20210101,00:15:00,00:20:00,3,19.52,52
4,"E4S 56,780",20210101,00:20:00,00:25:00,4,20.52,52
...,...,...,...,...,...,...,...
104838,"E4S 56,780",20211231,23:35:00,23:40:00,283,19.58,115
104839,"E4S 56,780",20211231,23:40:00,23:45:00,284,19.47,87
104840,"E4S 56,780",20211231,23:45:00,23:50:00,285,19.77,130
104841,"E4S 56,780",20211231,23:50:00,23:55:00,286,18.79,129


In [3]:
# Sort the DataFrame 'data_df' by columns "Date" and "Interval_5"
data_df.sort_values(["Date", "Interval_5"])
# Extract unique dates from the sorted DataFrame
days = np.unique(data_df[['Date']].values.ravel())
# Calculate the total number of unique days
ndays = len(days)
# Group the DataFrame 'data_df' by the "Date" column
day_subsets_df = data_df.groupby(["Date"])
# Define the total number of 5-minute intervals in a day
nintvals = 288

In [4]:
# Create a matrix 'vectorized_day_dataset' filled with NaN values
vectorized_day_dataset = np.zeros((ndays, nintvals))
vectorized_day_dataset.fill(np.nan)
# Loop through each unique day
for i in range(0, ndays):
    # Get the DataFrame corresponding to the current day
    df_t = day_subsets_df.get_group(days[i])
    # Loop through each row in the current day's DataFrame
    for j in range(len(df_t)):
        # Get the current day's DataFrame
        df_t = day_subsets_df.get_group(days[i])
        # Extract the "Interval_5" and "flow" values and populate 'vectorized_day_dataset'
        vectorized_day_dataset[i, df_t.iloc[j]["Interval_5"]] = df_t.iloc[j]["flow"]
# Print the resulting 'vectorized_day_dataset' and the size of vector
print(vectorized_day_dataset)
print(vectorized_day_dataset.shape)

[[ 39.  18.  26. ...  32.  39.  34.]
 [ 30.  32.  27. ...  44.  41.  39.]
 [ 36.  44.  52. ...  50.  45.  23.]
 ...
 [ 20.  34.  31. ...  38.  42.  36.]
 [ 36.  40.  25. ...  38.  56.  35.]
 [ 33.  32.  34. ... 130. 129. 117.]]
(365, 288)


In [5]:
# print the number of days with missing value
nans_per_day = np.sum(np.isnan(vectorized_day_dataset),1)
print('number of days with missing value:',np.size(np.where(nans_per_day > 0),1))

number of days with missing value: 28


In [6]:
# Drop the days with missing valus
vectorized_day_dataset_no_nans = vectorized_day_dataset[np.where(nans_per_day == 0)[0],:]
# days_not_nans = days[np.where(nans_per_day == 0)[0]]
print(vectorized_day_dataset_no_nans.shape)

(337, 288)


Assignment task: Compare the performance of PCA and kernel PCA for dimensionality reduction in dataset 1.

In [7]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, KernelPCA
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt

In [9]:
data = pd.DataFrame(vectorized_day_dataset_no_nans)
scaler = StandardScaler()
data_normalized = scaler.fit_transform(data)

In [12]:
# Perform PCA for dimensionality reduction
pca = PCA(n_components=10)  # Adjust the number of components (dimensions) as needed
data_pca = pca.fit_transform(data_normalized)


In [13]:
# Assess Performance of PCA
# Investigate Explained Variance
explained_variance = pca.explained_variance_ratio_
print(f'Explained Variance Ratio: {explained_variance}')

Explained Variance Ratio: [0.31725246 0.19340224 0.08298293 0.05444537 0.04555043 0.03011985
 0.02071443 0.01585592 0.01430133 0.01228154]


In [25]:
#  Perform kernel PCA for dimensionality reduction

kernel_list=['linear','rbf','poly','sigmoid']

df_results=pd.DataFrame()

for i in kernel_list:
  kpca = KernelPCA(kernel=i, n_components=10, fit_inverse_transform=True)
  data_kpca=kpca.fit_transform(data_normalized)
  explained_variance = np.var(data_kpca, axis=0)
  explained_variance_ratio = explained_variance / np.sum(explained_variance)
  df_results[i]=explained_variance_ratio

df_results

,linear,rbf,poly,sigmoid
0,0.403164,0.305813,0.526667,0.440166
1,0.245775,0.205979,0.188305,0.247148
2,0.105455,0.123078,0.088426,0.090422
3,0.069189,0.078553,0.047391,0.071106
4,0.057885,0.074426,0.036616,0.050970
5,0.038276,0.054230,0.029567,0.028969
6,0.026324,0.046548,0.025189,0.024109
7,0.020150,0.043585,0.021779,0.018684
8,0.018174,0.036348,0.018718,0.014673
9,0.015607,0.031440,0.017342,0.013753


In [29]:
df_results[pca]=pca.explained_variance_ratio_
df_results_T=df_results.T
df_results_T['sum']=df_results_T.apply(lambda x: x.sum(), axis=1)
df_results_T

,0,1,2,3,4,5,6,7,8,9,sum
linear,0.403164,0.245775,0.105455,0.069189,0.057885,0.038276,0.026324,0.020150,0.018174,0.015607,1.000000
rbf,0.305813,0.205979,0.123078,0.078553,0.074426,0.054230,0.046548,0.043585,0.036348,0.031440,1.000000
poly,0.526667,0.188305,0.088426,0.047391,0.036616,0.029567,0.025189,0.021779,0.018718,0.017342,1.000000
sigmoid,0.440166,0.247148,0.090422,0.071106,0.050970,0.028969,0.024109,0.018684,0.014673,0.013753,1.000000
PCA(n_components=10),0.317252,0.193402,0.082983,0.054445,0.045550,0.030120,0.020714,0.015856,0.014301,0.012282,0.786906
